In [10]:
#!/usr/bi#!/usr/bin/env python3
# -*- coding: utf-8 -*-


# import outputs from cleaning and manipulation modules
from manipulation import *


# import libraries and functions 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler 

In [11]:
# set df = whatever the final output of our dataframe is 

df = HousePriceDF


# lists and sublists of numeric and categorical variables 

numeric_all = ['GrLivArea', 'LotFrontage','LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 
            'LandSlope', 'OverallQual','OverallCond' , 'YearBuilt', 'YearRemodAdd',
           'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
           'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'HeatingQC', 
           'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 
           'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
           'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 
           'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 
           'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal' , 'MoSold', 'YrSold', 
           'SaleCondition', 'YrSinceRm', 'Pool', 'Misc', 'BsmtFinTotSF', 'TotalSF']

categorical_all  = ['MSSubClass', 'MSZoning', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
               'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st' , 'Exterior2nd', 'MasVnrType',
               'Foundation', 'Heating', 'GarageType','SaleType' , 'LotConfig']

In [12]:
# note the winners of the first round of Lasso were all numeric 
# set X = df[lasso1]
lasso1 = ['YearBuilt', 'YrSinceRm', 'TotalSF', 'BsmtUnfSF' , 'WoodDeckSF', 'GarageArea', '2ndFlrSF', 
          'ScreenPorch', 'GrLivArea', 'BsmtFinSF1', 'MasVnrArea', 'LotArea']

In [40]:
# winners for Lasso without log(y)
lasso2 =['Condition2_Pos', 'Neighborhood_GrnHill', 'Neighborhood_StoneBr', 'Pool', 'Exterior1st_VinylSd', 'Utilities', 
         'MSSubClass_150', 'RoofMatl_wood', 'MSSubClass_50', 'GarageType_None', 'MSSubClass_75', 'SaleType_New', 
         'Neighborhood_NridgHt', 'MSSubClass_30', 'Exterior1st_Wood', 'Neighborhood_NPkVill', 'MSSubClass_20', 
         'HouseStyle_SLvl', 'MSSubClass_40', 'Exterior2nd_VinylSd']

In [52]:
combo = lasso1 + lasso2

In [ ]:
#log(y)
y = np.log(df['SalePrice'])

In [41]:
# normal y
y = df['SalePrice']

In [42]:
# input which subset of numeric & categorical variables to be used 
numeric = numeric_all 

categorical = categorical_all 

In [43]:
# DO NOT TOUCH THIS CODE
dummies = pd.get_dummies(df[categorical], drop_first=True)

X = pd.concat([df[numeric], dummies], axis = 1)

In [46]:
# filtra per trovare in X le caratteristiche che combaciano con lasso2
X = X[lasso2]

In [47]:
X.shape

(2580, 20)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [21]:
# initiate and fit a full linear model 
# with log(y): R^2 = -29.784
# with y: -44168.576 

lm = LinearRegression()
lm.fit(X_train, y_train)
print('The Liner R^2 is equal to %.3f' %(lm.score(X_test, y_test)))
#print('The Linear intercept is %.3f' %(lm.intercept_))
#print('The Linear slopes are %s' %(lm.coef_))

The Liner R^2 is equal to -44168.576


In [22]:
# initiate Ridge
# with y = 0.901

rdg = Ridge()
rdg.fit(X_train, y_train)
print('The Ridge R^2 is equal to %.3f' %(rdg.score(X_test, y_test)))
#print('The Ridge intercept is %.3f' %(rdg.intercept_))
#print('The Ridge slopes are %s' %(rdg.coef_))

The Ridge R^2 is equal to 0.901


In [23]:
# initiate Lasso 
# with y:0.899
# list from Lasso (Condition2_Pos, Neighborhood_GrnHill, Neighborhood_StoneBr, Pool, Exterior1st_VinylSd, Utilities, 
# MSSubClass_150, RoofMatl_wood, MSSubClass_50, GarageType_None, MSSubClass_75, SaleType_New, Neighborhood_NridgHt, MSSubClass_30
# Exterior1st_Wood, Neighborhood_NPkVill, MSSubClass_20, HouseStyle_SLvl, MSSubClass_40, Exterior2nd_VinylSd)

ls = Lasso()
ls.fit(X_train, y_train)
print('The Lasso R^2 is equal to %.3f' %(ls.score(X_test, y_test)))
#print('The Lasso intercept is %.3f' %(?))
#print('The Lasso slopes are %s' %(?))
print('The Lasso coefs are %s' %(ls.coef_))

coefs = abs(ls.coef_)

features = pd.Series(coefs, index = X.columns)
features.sort_values(ascending=False).head(20)

The Lasso R^2 is equal to 0.899
The Lasso coefs are [ 9.30379163e+01  1.01712369e+01  4.45951069e-01  1.09794131e+04
  5.91354368e+02  4.17946177e+03 -2.54139566e+04 -3.88800143e+03
  8.28022170e+03  5.38277049e+03  3.35468991e+02  4.45007294e+02
  3.77625789e+01  8.01623695e+03  2.50395655e+02  4.01506080e+03
 -9.11110033e+03  4.51068567e+03  9.05090745e+02  2.91584318e+01
  2.21627677e+02  2.56162813e+01  7.88813040e+00  5.02854802e+00
  1.19435030e+03 -1.60922984e+03 -1.57495816e+03 -3.84374509e+01
 -3.40811073e+01 -5.74695264e+01  2.78171300e+03  2.18747240e+03
  1.79148196e+03  2.73838536e+03 -4.95183835e+03 -7.17641947e+03
  5.56874376e+03  9.90220857e+02 -4.69651765e+03  4.88612365e+03
 -6.36214290e+02 -2.35002687e+01  1.01183262e+03  3.65441054e+03
  1.95522394e+01  2.64690479e+03 -1.02358271e+02 -2.64102090e+01
  8.47827094e+00  3.57744565e+00  8.09482110e-01 -1.34975045e+00
  4.13131439e+01 -7.79732095e+01  7.35196472e-01 -4.70191442e+02
 -8.86515069e+02  1.12139507e+04  4.44

C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.801e+11, tolerance: 9.449e+08
  model = cd_fast.enet_coordinate_descent(


Condition2_Pos          85296.403490
Neighborhood_GrnHill    80193.620505
Neighborhood_StoneBr    38528.528935
Pool                    29879.888467
Exterior1st_VinylSd     26785.197102
Utilities               25413.956643
MSSubClass_150          24747.687045
RoofMatl_wood           24131.816396
MSSubClass_50           23458.909854
GarageType_None         22923.045758
MSSubClass_75           22059.933266
SaleType_New            21580.604686
Neighborhood_NridgHt    21446.547233
MSSubClass_30           21115.692684
Exterior1st_Wood        20814.639018
Neighborhood_NPkVill    20240.510230
MSSubClass_20           20060.478486
HouseStyle_SLvl         19923.945435
MSSubClass_40           19605.544854
Exterior2nd_VinylSd     19417.873178
dtype: float64

In [24]:
# initiate ElasticNet
# with y: 0.874 

alpha = 1.0
l1_ratio = 0.5
eln = ElasticNet()
eln.set_params(alpha=alpha, l1_ratio= l1_ratio)
eln.fit(X_train, y_train)
print('The ElasticNet R^2 is equal to %.3f' %(eln.score(X_test, y_test)))
#print('The ElasticNet intercept is %.3f' %(?))
#print('The ElasticNet slopes are %s' %(?))

The ElasticNet R^2 is equal to 0.874


C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.694e+11, tolerance: 9.449e+08
  model = cd_fast.enet_coordinate_descent(


In [25]:
# useing GridSearch for Lasso optimization 
# with y: 'model__alpha': 9.9

pipe = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])


gs = GridSearchCV(pipe,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="r2",verbose=3
                      )


gs.fit(X_train,y_train)
print('The GridSearch R^2 is equal to %.3f' %(gs.score(X_test, y_test))) 


gs.best_params_

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END ..................model__alpha=0.1;, score=0.818 total time=   0.0s


C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.311e+11, tolerance: 7.518e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.518e+11, tolerance: 7.526e+08
  model = cd_fast.enet_coordinate_descent(


[CV 2/5] END ..................model__alpha=0.1;, score=0.903 total time=   0.1s


C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.513e+11, tolerance: 7.290e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.010e+11, tolerance: 7.752e+08
  model = cd_fast.enet_coordinate_descent(


[CV 3/5] END ..................model__alpha=0.1;, score=0.896 total time=   0.1s
[CV 4/5] END ..................model__alpha=0.1;, score=0.818 total time=   0.1s


...

[CV 3/5] END ..................model__alpha=9.9;, score=0.896 total time=   0.0s
[CV 4/5] END ..................model__alpha=9.9;, score=0.855 total time=   0.1s


C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.836e+09, tolerance: 7.752e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.309e+09, tolerance: 7.710e+08
  model = cd_fast.enet_coordinate_descent(


[CV 5/5] END ..................model__alpha=9.9;, score=0.868 total time=   0.0s
The GridSearch R^2 is equal to 0.899


C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.372e+09, tolerance: 9.449e+08
  model = cd_fast.enet_coordinate_descent(


{'model__alpha': 9.9}

In [26]:
# input which subset of numeric & categorical variables to be used for selected with Lasso(log(y))
numeric = numeric_all 

categorical = lasso1 

In [27]:
# trying again with X selected with Lasso
dummies = pd.get_dummies(df[categorical], drop_first=True)

X = pd.concat([df[numeric], dummies], axis = 1)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [29]:
# linear model with lassos
# with y: 0.884

lm = LinearRegression()
lm.fit(X_train, y_train)
print('The Liner R^2 is equal to %.3f' %(lm.score(X_test, y_test)))
#print('The Linear intercept is %.3f' %(lm.intercept_))
#print('The Linear slopes are %s' %(lm.coef_))

The Liner R^2 is equal to 0.884


In [30]:
# initiate Ridge with Lassos
# with y = 0.884

rdg = Ridge()
rdg.fit(X_train, y_train)
print('The Ridge R^2 is equal to %.3f' %(rdg.score(X_test, y_test)))
#print('The Ridge intercept is %.3f' %(rdg.intercept_))
#print('The Ridge slopes are %s' %(rdg.coef_))

The Ridge R^2 is equal to 0.884


In [31]:
# initiate Lasso with lassos
# with y: 0.885

ls = Lasso()
ls.fit(X_train, y_train)
print('The Lasso R^2 is equal to %.3f' %(ls.score(X_test, y_test)))
#print('The Lasso intercept is %.3f' %(?))
#print('The Lasso slopes are %s' %(?))
print('The Lasso coefs are %s' %(ls.coef_))

coefs = abs(ls.coef_)

features = pd.Series(coefs, index = X.columns)
features.sort_values(ascending=False).head(20)

The Lasso R^2 is equal to 0.885
The Lasso coefs are [ 1.06725708e+02  1.62516905e+02  6.07966060e-01  1.74297453e+04
 -3.43434783e+03  2.53727522e+02 -3.44166642e+04 -4.79742077e+03
  9.63673054e+03  5.39581971e+03  5.72214850e+02  1.33201876e+02
  2.95377415e+01  1.19440149e+04 -1.03170534e+03  5.66196039e+03
 -9.58349476e+03  2.63468125e+03  3.85863043e+02  2.34633221e+01
 -9.46644344e+01  2.86268524e+01  2.91311164e+00  2.35896172e+00
  2.45325084e+03 -5.83081431e+03 -4.66117252e+03 -3.84775364e+01
 -3.88701157e+01 -8.60867048e+01  1.76811539e+03 -8.21146132e+02
 -4.63431053e+02  1.27887664e+03 -5.13640712e+03 -2.07692969e+04
  6.39476509e+03  1.78421575e+03 -4.83561464e+03  3.40299753e+03
  1.39031960e+03 -2.78975076e+01 -3.65667416e+02  7.25331716e+02
  1.60530860e+01 -2.84029889e+03 -2.29764523e+03 -1.79673511e+03
 -6.15823013e+00 -2.00543347e+01  7.11379208e+00  6.37935460e+00
  4.59050626e+01 -1.31105738e+02  8.19053880e-01 -2.60804830e+02
 -6.07965143e+02 -1.73701544e+03  3.20

C:\Users\Francesco\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.964e+11, tolerance: 9.449e+08
  model = cd_fast.enet_coordinate_descent(


Pool            52037.704166
Utilities       34416.664181
KitchenAbvGr    20769.296919
Street          17429.745346
ExterQual       11944.014898
OverallQual      9636.730542
BsmtCond         9583.494762
KitchenQual      6394.765086
CentralAir       5830.814305
BsmtQual         5661.960385
OverallCond      5395.819711
BedroomAbvGr     5136.407119
Functional       4835.614642
LandSlope        4797.420765
Electrical       4661.172520
Misc             3775.165351
LotShape         3434.347826
Fireplaces       3402.997527
GarageQual       2840.298892
BsmtExposure     2634.681251
dtype: float64

In [49]:
# initiate linear model with selection from Lasso without log(y)
# with y: 0.439 

lm = LinearRegression()
lm.fit(X_train, y_train)
print('The Liner R^2 is equal to %.3f' %(lm.score(X_test, y_test)))
#print('The Linear intercept is %.3f' %(lm.intercept_))
#print('The Linear slopes are %s' %(lm.coef_))

The Liner R^2 is equal to 0.439


In [50]:
# Ridge with selection from Lasso without log(y)
# with y = 0.434

rdg = Ridge()
rdg.fit(X_train, y_train)
print('The Ridge R^2 is equal to %.3f' %(rdg.score(X_test, y_test)))
#print('The Ridge intercept is %.3f' %(rdg.intercept_))
#print('The Ridge slopes are %s' %(rdg.coef_))

The Ridge R^2 is equal to 0.434


In [51]:
# initiate with selection from Lasso without log(y)
# with y: 0.874 

alpha = 1.0
l1_ratio = 0.5
eln = ElasticNet()
eln.set_params(alpha=alpha, l1_ratio= l1_ratio)
eln.fit(X_train, y_train)
print('The ElasticNet R^2 is equal to %.3f' %(eln.score(X_test, y_test)))
#print('The ElasticNet intercept is %.3f' %(?))
#print('The ElasticNet slopes are %s' %(?))

The ElasticNet R^2 is equal to 0.115


In [54]:
# input which subset of numeric & categorical variables to be used for selected with Lasso(log(y))
numeric = numeric_all 

categorical = categorical_all

In [57]:
# trying again with X selected with Lasso
dummies = pd.get_dummies(df[categorical], drop_first=True)

X = pd.concat([df[numeric], dummies], axis = 1)

X = X[lasso2]

KeyError: "['Condition2_Pos', 'Neighborhood_GrnHill', 'Neighborhood_StoneBr', 'Exterior1st_VinylSd', 'MSSubClass_150', 'RoofMatl_wood', 'MSSubClass_50', 'GarageType_None', 'MSSubClass_75', 'SaleType_New', 'Neighborhood_NridgHt', 'MSSubClass_30', 'Exterior1st_Wood', 'Neighborhood_NPkVill', 'MSSubClass_20', 'HouseStyle_SLvl', 'MSSubClass_40', 'Exterior2nd_VinylSd'] not in index"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
# linear model with lassos
# with y: 0.884

lm = LinearRegression()
lm.fit(X_train, y_train)
print('The Liner R^2 is equal to %.3f' %(lm.score(X_test, y_test)))
#print('The Linear intercept is %.3f' %(lm.intercept_))
#print('The Linear slopes are %s' %(lm.coef_))